In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime

## fetch all berlin weihnachtsmarkt links

In [2]:
url = 'https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/'
res = requests.get(url)
print("Successfully fetched data")

soup = BeautifulSoup(res.content, 'html.parser')
class_name = 'eintraege'
div_section = soup.find('div', class_=class_name)

links = div_section.find_all('a')
base_url = 'https://www.deutsche-weihnachtsmaerkte.de/'
wmlinks = [base_url + link['href'] for link in links]


Successfully fetched data


## getting dates and opening hours for each weihnachtsmarkt

In [3]:
dc = []
url = wmlinks[1]
print(url)
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-vor-dem-schloss-charlottenburg-1627.html


In [4]:
td_class_dates = 'oeffnungszeiten'
td_class_day = 'oeffnungszeiten-tag'
td_class_hours = 'oeffnungszeiten-zeit'
dc = {
    "name":[],
    "address":[],
    "url":[],
    "dates":[],
    "opening_hours":[]
}

for url in wmlinks:
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')

    name = soup.find('h1', class_="underlined").text

    address = soup.find('div', class_="ort").find("p").text.replace("\n", " ")
    
    div_section = soup.find('div', class_=td_class_dates)
    dates = div_section.find('p')
    dates = dates.text if dates else None

    opening_hours = []
    
    rows = soup.select('table.oeffnungszeiten-tabelle tr')

    for row in rows:
        tag = row.select_one('.oeffnungszeiten-tag').get_text(strip=True)
        zeit = row.select_one('.oeffnungszeiten-zeit').get_text(strip=True)
        opening_hours.append((tag, zeit))

    dc["name"].append(name)
    dc["address"].append(address)
    dc["url"].append(url)
    dc["dates"].append(dates)
    dc["opening_hours"].append(opening_hours)
    time.sleep(1)
    
    

https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-evangelisches-johannesstift-berlin-1585.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-vor-dem-schloss-charlottenburg-1627.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtszeit-rotes-rathaus-berlin-352.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lichtenberger-lichtermarkt-berlin-378.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/advents-oekomarkt-gruene-liga-berlin-379.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lucia-weihnachtsmarkt-kulturbrauerei-berlin-380.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/daenischer-adventsbasar-berlin-381.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/schwedischer-weihnachtsbasar-berlin-wilmersdorf-382.html
https://www.deutsche-weihnachtsmaerkte.de/weihnachtsma

In [7]:
pd.DataFrame(dc).to_csv('collected_data.csv', index=False)


## data cleaning and creating full calendar

In [2]:
google_maps = {
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-evangelisches-johannesstift-berlin-1585.html": "https://maps.app.goo.gl/7nGJednvX2cEgioLA",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-vor-dem-schloss-charlottenburg-1627.html": "https://maps.app.goo.gl/rTQEKsMdZLCsW42L6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtszeit-rotes-rathaus-berlin-352.html": "https://maps.app.goo.gl/QBX4aYtKjEMj2cKu9",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lichtenberger-lichtermarkt-berlin-378.html": "https://maps.app.goo.gl/mnoS87pZNXBHE1Wf9",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/advents-oekomarkt-gruene-liga-berlin-379.html": "https://maps.app.goo.gl/CQa6BuStrVrr1VVz6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lucia-weihnachtsmarkt-kulturbrauerei-berlin-380.html": "https://maps.app.goo.gl/dFA6WsbdFtKJbDpAA",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/daenischer-adventsbasar-berlin-381.html": "https://maps.app.goo.gl/GVvjWt1vcxkrsSMm7",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/schwedischer-weihnachtsbasar-berlin-wilmersdorf-382.html": "https://maps.app.goo.gl/grkiqMzt7QWVxY827",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/schoeneberger-weihnachtsmarkt-berlin-383.html": "https://maps.app.goo.gl/E9FJNMa3o2nHt5bq6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lichtenrader-lichtermarkt-berlin-386.html": "https://maps.app.goo.gl/muSYA8GqnZSf7E6s5",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/nordische-maerchenweihnacht-schloss-britz-387.html": "https://maps.app.goo.gl/uKEZnV3f7LfrYjJB7",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-spaethsche-baumschulen-berlin-389.html": "https://maps.app.goo.gl/7iCTSzQjZi2oU3KdA",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/lichtermarkt-christuskirche-berlin-390.html": "https://maps.app.goo.gl/BQHR5ojMno3Te13D6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-frohnau-393.html": "https://maps.app.goo.gl/StJyT3ugduKwYAC77",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-gedaechtniskirche-berlin-1628.html": "https://maps.app.goo.gl/DjLcxHMBajrQ1zW58",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtshaus-villa-donnersmarck-berlin-397.html": "https://maps.app.goo.gl/fCPYh2FN8wJoRcsJ6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-grunewaldkirche-berlin-399.html": "https://maps.app.goo.gl/UuqyaWtR4z54yt4z8",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-st-michaels-heim-grunewald-berlin-400.html": "https://maps.app.goo.gl/8YTjikBNhHrX54K59",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/domaenen-adventsmarkt-berlin-dahlem-401.html": "https://maps.app.goo.gl/GBHQqZyPmcQthZHXA",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/gruenauer-weihnachtsmarkt-berlin-2690.html": "https://maps.app.goo.gl/AsgP9Jvh44r1kABt8",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachten-spree-all-inclusive-weihnachtsmarkt-berlin-3182.html": "https://maps.app.goo.gl/9GNV78jQoBvmV9vE7",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/winterzauber-berlin-weihnachtsmarkt-3079.html": "https://maps.app.goo.gl/MSf5pSxjkbM82dU79",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/christsternmarkt-lankwitz-kirche-1392.html": "https://maps.app.goo.gl/prfB1t3Dy7nDs9aG6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtszauber-gendarmenmarkt-1477.html": "https://maps.app.goo.gl/25vXgYXWhH3bXY9y5",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-jagdschloss-grunewald-1512.html": "https://maps.app.goo.gl/gPDjMsrZt5tmKDno6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/berlin-spandau-spandauer-weihnachtsmarkt-1510.html": "https://maps.app.goo.gl/TTm9PssysEHAKEiN9",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-auf-lehmanns-bauernhof-berlin-1511.html": "https://maps.app.goo.gl/W3K2WXkqrecarieW9",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/winterwelt-potsdamer-platz-berlin-1513.html": "https://maps.app.goo.gl/5abyHXA2PRygDHae6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/veganer-weihnachtsmarkt-berlin-2667.html": "https://maps.app.goo.gl/nMMpK5ifUkxe9DvR8",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/vegan-vegetarischer-weihnachtsmarkt-the-green-market-berlin-2668.html": "https://maps.app.goo.gl/YhwACjRHANbWvwgU6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachts-basar-lichthof-auswaertiges-amt-berlin-3367.html": "https://maps.app.goo.gl/icK5tu6ZkvEEfdPc6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/adventsbasar-albert-schweitzer-stiftung-wohnen-betreuen-berlin-3379.html": "https://maps.app.goo.gl/gjLA1b8Ra3ufZXuB7",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/adventsmarkt-freie-evangelische-gemeinde-berlin-adlershof-3387.html": "https://maps.app.goo.gl/ZApUfKCtguvQdLQJ9",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/altglienicker-adventsmarkt-berlin-3266.html": "https://maps.app.goo.gl/1JunUEkxpzzsch616",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/kunstadvent-berlin-friedrichshagen-3226.html": "https://maps.app.goo.gl/WCP7oLBeJLVeUEi5A",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/historischer-weihnachtsmarkt-berlin-2781.html": "https://maps.app.goo.gl/DWzwbmC3HpBUB5zU6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachtsmarkt-koepenick-1503.html": "https://maps.app.goo.gl/VNciZPRjPgDDxcht8",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/winterzauber-weihnachtsmarkt-strandbad-wendenschloss-berlin-koepenick-3249.html": "https://maps.app.goo.gl/aTi6kGg3gAtWTTko7",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/huettenzauber-maerchenwald-koepenick-berlin-3413.html": "https://maps.app.goo.gl/oFRaA89DeUkjuaYr8",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/alt-rixdorfer-weihnachtsmarkt-1475.html": "https://maps.app.goo.gl/gDihRbFGJsUKsPo17",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/kladower-christkindlmarktes-2807.html": "https://maps.app.goo.gl/x1sYoJDfzenFvTFh6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/weihnachten-westend-2808.html": "https://maps.app.goo.gl/YZkqyDL79AdqhCJX6",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/rudower-adventsmeile-2809.html": "https://maps.app.goo.gl/UAxG8k6hmt1MNavX7",
    "https://www.deutsche-weihnachtsmaerkte.de/weihnachtsmarkt/in/berlin/christmas-avenue-berlin-3001.html": "https://maps.app.goo.gl/E6xGPqh33KRnzDqD8",
}

In [3]:

LOAD_LOCAL_CSV = True
if LOAD_LOCAL_CSV:
    df = pd.read_csv('collected_data.csv')
    df['dates'] = df['dates'].apply(lambda x: str(x) if x!='nan' else None)
    df.at[8, 'dates'] = None
    df['opening_hours'] = df['opening_hours'].apply(lambda x: eval(x))
else:
    df = pd.DataFrame(dc)

df['start_date'] = df['dates'].apply(lambda x: '' if x is None else x.split('-')[0].strip())
df['start_date'] = df['start_date'].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y') if x else None)
df['end_date'] = df['dates'].apply(lambda x: '' if x is None else x.split('-')[1].strip())
df['end_date'] = df['end_date'].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y') if x else None)
df['days_open'] = df.apply(lambda row: (row['end_date'] - row['start_date']).days, axis=1)
df['days'] = df['opening_hours'].apply(lambda x: [d[0] for d in x])

german_days = ["MO", "DI", "MI", "DO", "FR", "SA", "SO"]
german_day_mapper = {
    german_days[i]:i for i in range(len(german_days))
}

df['day_indexes'] = df['days'].apply(lambda x: set([german_day_mapper[d] for d in x]))
df['google_maps'] = df['url'].apply(lambda x: google_maps[x] if x in google_maps else print(x))
df.sort_values(by='days_open', ignore_index=True)

,name,address,url,dates,opening_hours,start_date,end_date,days_open,days,day_indexes,google_maps
0,Weihnachtsmarkt Evangelisches Johannesstift Be...,Schönwalder Allee 26 13587 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 03.12.2023,"[(SO, 12:00 - 18:00 Uhr)]",2023-12-03,2023-12-03,0.0,[SO],{6},https://maps.app.goo.gl/7nGJednvX2cEgioLA
1,Altglienicker Adventsmarkt in Berlin,Rund um das Evangelische Gemeindezentrum Roses...,https://www.deutsche-weihnachtsmaerkte.de/weih...,10.12.2023 - 10.12.2023,"[(SO, 14:00 - 19:00 Uhr)]",2023-12-10,2023-12-10,0.0,[SO],{6},https://maps.app.goo.gl/1JunUEkxpzzsch616
2,Adventsmarkt der Freie evangelische Gemeinde i...,Rund um die Schwedenkirche Handjerystr. 29/31 ...,https://www.deutsche-weihnachtsmaerkte.de/weih...,02.12.2023 - 02.12.2023,"[(SA, 15:00 - 19:00 Uhr)]",2023-12-02,2023-12-02,0.0,[SA],{5},https://maps.app.goo.gl/ZApUfKCtguvQdLQJ9
3,Adventsbasar der Albert Schweitzer Stiftung – ...,Stiftungsgelände der Albert Schweitzer Stiftun...,https://www.deutsche-weihnachtsmaerkte.de/weih...,02.12.2023 - 02.12.2023,"[(SA, 15:00 - 18:00 Uhr)]",2023-12-02,2023-12-02,0.0,[SA],{5},https://maps.app.goo.gl/gjLA1b8Ra3ufZXuB7
4,Grünauer Weihnachtsmarkt Berlin,Grünauer Friedenskirche Don-Ugoletti-Platz 125...,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 03.12.2023,"[(SO, 13:00 - 20:00 Uhr)]",2023-12-03,2023-12-03,0.0,[SO],{6},https://maps.app.goo.gl/AsgP9Jvh44r1kABt8
5,Weihnachtsmarkt Grunewaldkirche Berlin,Grunewaldkirche Furtwänglerstr. 5 14193 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,03.12.2023 - 03.12.2023,"[(SO, 10:00 - 19:00 Uhr)]",2023-12-03,2023-12-03,0.0,[SO],{6},https://maps.app.goo.gl/UuqyaWtR4z54yt4z8
6,Weihnachtshaus Villa Donnersmarck Berlin,Villa Donnersmarck Schädestr. 9 - 13 14163 Be...,https://www.deutsche-weihnachtsmaerkte.de/weih...,17.12.2023 - 17.12.2023,"[(SO, 15:00 - 18:00 Uhr)]",2023-12-17,2023-12-17,0.0,[SO],{6},https://maps.app.goo.gl/fCPYh2FN8wJoRcsJ6
7,Lichtermarkt Christuskirche Berlin,Christuskirche Firlstraße 12459 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,02.12.2023 - 02.12.2023,"[(SA, 14:00 - 18:00 Uhr)]",2023-12-02,2023-12-02,0.0,[SA],{5},https://maps.app.goo.gl/BQHR5ojMno3Te13D6
8,Christsternmarkt Lankwitz-Kirche,"Lankwitz-Kirche, Dreifaltigkeits-Gemeinde Gall...",https://www.deutsche-weihnachtsmaerkte.de/weih...,02.12.2023 - 02.12.2023,"[(SA, 13:00 - 18:00 Uhr)]",2023-12-02,2023-12-02,0.0,[SA],{5},https://maps.app.goo.gl/prfB1t3Dy7nDs9aG6
9,Dänischer Adventsbasar Berlin,Christianskirche Brienner Str. 12 10713 Berlin,https://www.deutsche-weihnachtsmaerkte.de/weih...,25.11.2023 - 25.11.2023,"[(SA, 12:00 - 22:00 Uhr)]",2023-11-25,2023-11-25,0.0,[SA],{5},https://maps.app.goo.gl/GVvjWt1vcxkrsSMm7


In [4]:
output = df[['name', 'address', 'dates', 'url', 'google_maps']].to_markdown()

with open("markets_info.md", "w") as f:
    f.write(output)

## Producing all markets calendar

In [5]:
def find_available_christmas_markets(given_date, df):
    available_market_indexes = []
    for i in range(len(df)):
        if df.at[i, 'start_date'] <= given_date <= df.at[i, 'end_date'] and given_date.weekday() in df.at[i, 'day_indexes']:
            available_market_indexes.append(i)
    return available_market_indexes
            
find_available_christmas_markets(datetime.datetime.now(), df)

[21, 27, 35]

In [6]:
file_output = ""

start_date = datetime.datetime.now()
start_date = datetime.datetime.combine(start_date.date(), datetime.time.min)
end_date = datetime.datetime(2023, 12, 31)
step = datetime.timedelta(days=1)

current_date = start_date
while current_date <= end_date:
    file_output+= "## " +current_date.strftime("%Y-%m-%d, %A") + "\n"

    markets = find_available_christmas_markets(current_date, df)
    for i in markets:
        file_output += f"- [{df.at[i, 'name']}]({df.at[i, 'url']}), address: [{df.at[i, 'address']}]({df.at[i, 'google_maps']})" + "\n"
    
    current_date += step

with open("calendar.md", "w") as f:
    f.write(file_output)

## Producing calendar for markets open for few days

In [7]:
short_markets_df = df[df['days_open'] <= 3].reset_index(drop=True)
file_output = ""

start_date = datetime.datetime.now()
start_date = datetime.datetime.combine(start_date.date(), datetime.time.min)
end_date = datetime.datetime(2023, 12, 31)
step = datetime.timedelta(days=1)

current_date = start_date
while current_date <= end_date:
    file_output+= "## " +current_date.strftime("%Y-%m-%d, %A") + "\n"

    markets = find_available_christmas_markets(current_date, short_markets_df)
    for i in markets:
        file_output += f"- [{short_markets_df.at[i, 'name']}]({short_markets_df.at[i, 'url']}), address: [{short_markets_df.at[i, 'address']}]({short_markets_df.at[i, 'google_maps']})" + "\n"
    
    current_date += step

with open("calendar_short_markets.md", "w") as f:
    f.write(file_output)